In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from functools import partial

# Load user data as before
def load_user_data(base_folder):
    user_data = {}
    for user_folder in os.listdir(base_folder):
        user_path = os.path.join(base_folder, user_folder)
        if os.path.isdir(user_path):
            session_data = {}
            for file in os.listdir(user_path):
                if file.endswith('.csv'):
                    file_path = os.path.join(user_path, file)
                    data = pd.read_csv(file_path)
                    if 'x' in data.columns and 'y' in data.columns:
                        session_data[file] = data[['x', 'y']]
            if session_data:
                user_data[user_folder] = session_data
    return user_data

train_data = load_user_data('training_files')

# WGAN-GP requires a more sophisticated setup

# Generator model
def build_generator(latent_dim):
    model = tf.keras.Sequential([
        layers.Dense(128, activation='relu', input_dim=latent_dim),
        layers.Dense(256, activation='relu'),
        layers.Dense(2, activation='linear')  # Output is two values (x, y)
    ])
    return model

# Discriminator model
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(2,)),
        layers.Dense(128, activation='relu'),
        layers.Dense(1)  # Output is a single value (no activation for WGAN)
    ])
    return model

# Gradient penalty
def gradient_penalty(discriminator, real_data, fake_data, batch_size):
    epsilon = tf.random.normal([batch_size, 1], 0.0, 1.0)
    interpolated_data = epsilon * real_data + (1 - epsilon) * fake_data
    with tf.GradientTape() as tape:
        tape.watch(interpolated_data)
        interpolated_output = discriminator(interpolated_data)
    gradients = tape.gradient(interpolated_output, interpolated_data)
    gradients_norm = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=[1]))
    gradient_penalty = tf.reduce_mean((gradients_norm - 1.0) ** 2)
    return gradient_penalty

# Training function
def train_wgan(generator, discriminator, data, epochs=100, batch_size=128, latent_dim=100, n_critic=5, gp_weight=10.0):
    generator_optimizer = RMSprop(learning_rate=0.00005)
    discriminator_optimizer = RMSprop(learning_rate=0.00005)

    for epoch in range(epochs):
        for _ in range(n_critic):
            real_data = data[np.random.randint(0, data.shape[0], batch_size)]
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            fake_data = generator(noise)

            with tf.GradientTape() as tape:
                d_real = discriminator(real_data)
                d_fake = discriminator(fake_data)
                gp = gradient_penalty(discriminator, real_data, fake_data, batch_size)
                d_loss = tf.reduce_mean(d_fake) - tf.reduce_mean(d_real) + gp_weight * gp

            gradients = tape.gradient(d_loss, discriminator.trainable_variables)
            discriminator_optimizer.apply_gradients(zip(gradients, discriminator.trainable_variables))

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        misleading_targets = -np.ones((batch_size, 1))

        with tf.GradientTape() as tape:
            g_loss = -tf.reduce_mean(discriminator(generator(noise)))

        gradients = tape.gradient(g_loss, generator.trainable_variables)
        generator_optimizer.apply_gradients(zip(gradients, generator.trainable_variables))

        if epoch % 100 == 0:
            print(f'Epoch: {epoch}, D Loss: {d_loss.numpy()}, G Loss: {g_loss.numpy()}')

# Generate fake data
def generate_fake_data(generator, latent_dim, num_samples):
    noise = np.random.normal(0, 1, (num_samples, latent_dim))
    fake_data = generator(noise)
    return fake_data

In [ ]:
latent_dim = 100
output_folder = 'fake_data'
os.makedirs(output_folder, exist_ok=True)

for user, sessions in train_data.items():
    print(f"Training GAN for user: {user}")
    user_coordinates = pd.concat(sessions.values())[["x", "y"]].values
    generator = build_generator(latent_dim)
    discriminator = build_discriminator()

    train_wgan(generator, discriminator, user_coordinates)

    user_output_folder = os.path.join(output_folder, user)
    os.makedirs(user_output_folder, exist_ok=True)

    for session, data in sessions.items():
        num_samples = len(data)
        fake_data = generate_fake_data(generator, latent_dim, num_samples)
        fake_data_df = pd.DataFrame(fake_data.numpy(), columns=['x', 'y'])
        fake_data_df.to_csv(os.path.join(user_output_folder, session), index=False)